### Imports

#### Independent Imports

In [1]:
import random
import sys
import os
import cv2
import skimage.io
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

#### Location Dependent Imports

In [2]:
local = Path.home().group() == 'ademyanchuk'

In [3]:
if local:
    from chestxray.config import PANDA_PATH, PANDA_IMGS
    
    TEST_PATH = PANDA_IMGS
    ADD_DIR = Path("/home/ademyanchuk/Projects/ChestXRay/submit_panda")

In [4]:
if not local:
    ADD_DIR = Path("../input/panda0")

In [5]:
sys.path.append(str(ADD_DIR))

In [6]:
# Imports from dataset
from submit_patch import PatchModel
from submit_patch import CFG
from submit_patch import PatchTestDataset

In [7]:
if not local:
    from submit_patch import TEST_PATH, PANDA_PATH

### Setup

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [10]:
def seed_torch(seed=1982):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch()

In [11]:
os.listdir(PANDA_PATH)

['train_label_masks',
 'sample_submission.csv',
 'test.csv',
 'train_images',
 'folds.csv',
 'folds_db.csv',
 'hdf5',
 'train.csv']

In [12]:
TEST_PATH.exists()

True

### Data Loading

In [13]:
train = pd.read_csv(PANDA_PATH/'train.csv')
test = pd.read_csv(PANDA_PATH/'test.csv')
sample = pd.read_csv(PANDA_PATH/'sample_submission.csv')

In [14]:
train.head()

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0


In [15]:
test.head()

,image_id,data_provider
0,005700be7e06878e6605e7a5a39de1b2,radboud
1,005c6e8877caf724c600fdce5d417d40,karolinska
2,0104f76634ff89bfff1ef0804a95c380,radboud


In [16]:
sample.head()

,image_id,isup_grade
0,005700be7e06878e6605e7a5a39de1b2,0
1,005c6e8877caf724c600fdce5d417d40,0
2,0104f76634ff89bfff1ef0804a95c380,0


### Model

In [17]:
ls {ADD_DIR}/26-05-2020-08-27_loss.pth

/home/ademyanchuk/Projects/ChestXRay/submit_panda/26-05-2020-08-27_loss.pth


In [18]:
model = PatchModel(arch=CFG.arch, pretrained=False)

weights_path = ADD_DIR/"26-05-2020-08-27_loss.pth"
checkpoint = torch.load(weights_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

### Inference

In [19]:
def inference(model, test_loader, device):
    
    model.to(device)
    model.eval()
    
    preds = []

    for i, images in enumerate(test_loader):
            
        images = images.to(device)
            
        with torch.no_grad():
            y_preds = model(images)
            
        preds.append(y_preds.to('cpu').numpy().argmax(1))

    preds = np.concatenate(preds)
    
    return preds

In [20]:
def submit(sample):
    if TEST_PATH.exists():
        test_dataset = PatchTestDataset(sample, img_path=TEST_PATH)
        test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2)
        preds = inference(model, test_loader, device)
        sample['isup_grade'] = preds
    return sample

In [21]:
if local:
    sample = train.sample(50)
submission = submit(sample)
submission['isup_grade'] = submission['isup_grade'].astype(int)
submission.head()

,image_id,data_provider,isup_grade,gleason_score
5515,87c639ec5365c2d6d61581be6407c8b6,radboud,5,4+5
7977,c10eb86a927f359f5d1e573927f19945,radboud,1,3+3
9742,ec11cdbfe083f766ebc6a851353a1662,karolinska,1,3+4
611,0f83710916fcc2edf1a7a9a103fd48ac,radboud,2,3+4
1837,2e53a31a0727921e823b781914782036,karolinska,1,3+3


In [ ]:
submission.to_csv('submission.csv', index=False)